In [18]:
import os
import re

from pathlib import Path

In [19]:
# script to separate report (OK)
# script to clean 3 lines of introduction but there are some that only have 2 lines, so need to check for the 3rd line first word -> "pemeriksaan" (OK)

# script to check whether there is wrong report file in folder
# script to detect which "kesan" is the conclusion 

'''
26
TS Yang terhormat

Pemeriksaan radiografi pelvis proyeksi AP dengan hasil sebagai berikut :

Kedudukan tulang-tulang pelvis berubah
Tampak fraktur pada neck femur dextra dengan fragmen fraktur displaced ke cranial
Celah sendi coxae dan sakroiliaka kanan-kiri serta simfisis pubis dalam batas normal.
Jaringan lunak tak tampak lesi patologis.


Kesan:
Fraktur pada neck femur dextra

Terima kasih atas kerjasamanya,
55 pelvis
TS Yang terhormat

Pemeriksaan radiografi pelvis proyeksi AP dengan hasil sebagai berikut :

Kedudukan tulang-tulang femur dextra berubah
Tampak fraktur pada intertrochanter femur dextra dengan fragmen fraktur displaced minimal
Celah sendi coxae dan sakroiliaka kanan-kiri serta simfisis pubis dalam batas normal.
Jaringan lunak tak tampak lesi patologis.

Kesan:
Fraktur pada intertrochanter femur dextra

Terima kasih atas kerjasamanya,
56 pelvis
TS Yang terhormat

Pemeriksaan radiografi pelvis proyeksi AP dengan hasil sebagai berikut :

Kedudukan tulang-tulang pelvis baik.
Tak tampak fraktur, destruksi, dislokasi, maupun lesi litik/blastik lainnya.
Celah sendi coxae dan sakroiliaka kanan-kiri serta simfisis pubis dalam batas normal.
Jaringan lunak tak tampak lesi patologis.


Kesan:
Tak tampak kelainan radiologik.


Terima kasih atas kerjasamanya,

'''

'\n26\nTS Yang terhormat\n\nPemeriksaan radiografi pelvis proyeksi AP dengan hasil sebagai berikut :\n\nKedudukan tulang-tulang pelvis berubah\nTampak fraktur pada neck femur dextra dengan fragmen fraktur displaced ke cranial\nCelah sendi coxae dan sakroiliaka kanan-kiri serta simfisis pubis dalam batas normal.\nJaringan lunak tak tampak lesi patologis.\n\n\nKesan:\nFraktur pada neck femur dextra\n\nTerima kasih atas kerjasamanya,\n55 pelvis\nTS Yang terhormat\n\nPemeriksaan radiografi pelvis proyeksi AP dengan hasil sebagai berikut :\n\nKedudukan tulang-tulang femur dextra berubah\nTampak fraktur pada intertrochanter femur dextra dengan fragmen fraktur displaced minimal\nCelah sendi coxae dan sakroiliaka kanan-kiri serta simfisis pubis dalam batas normal.\nJaringan lunak tak tampak lesi patologis.\n\nKesan:\nFraktur pada intertrochanter femur dextra\n\nTerima kasih atas kerjasamanya,\n56 pelvis\nTS Yang terhormat\n\nPemeriksaan radiografi pelvis proyeksi AP dengan hasil sebagai beriku

In [22]:
def txt_file(filepath):
    if filepath.endswith('TOTAL.txt'):
        pass
    else:
        return filepath.lower().endswith('.txt')

def separate_report(text):
    report = {}
    current_report = []
    num = 1
    for sentence in text.split('\n'):
        for word in sentence.split():
            # get report number for dict key
            if word.isdigit():
                num = word
            # if word is a digit (means new report) and it is not the first digit (report)
            if word.isdigit() and word != text.split()[0]:
                # combine all the words into one report and add it to dict
                final_report = ' '.join(current_report)
                final_report = '\n'.join([f.lstrip(' ') for f in final_report.splitlines()])
                report[num] = final_report
                # clear list and append new digit (report)
                current_report = []
                current_report.append(word)
            # elif last word in report, add the final report to the dict
            elif word == text.split()[-1]:
                current_report.append(word)
                final_report = ' '.join(current_report)
                final_report = '\n'.join([f.lstrip(' ') for f in final_report.splitlines()])
                report[num] = final_report
            else:
                current_report.append(word)
        current_report.append('\n')
    # {report_num_1: text, report_num_2: text}
    return report

def makedir(path):
    Path(path).mkdir(parents=True, exist_ok=True)
    # change permission into read, write, execute
    Path(path).chmod(0o777)

def generate_separate_report(text, base_dir, filename):
    report = separate_report(text)
    # ex path: base_dir/body_part/series/filename
    # ex filename: body_part-series+reportnum ??
    body_part = Path(filename).parent.parent.name
    for key, val in report.items():
        base_new_filename = f'{body_part}-num{key}_series_1001'
        new_filename = f'{base_new_filename}.txt' # need to confirm number and series
        new_filepath = os.path.join(base_dir, body_part, base_new_filename, new_filename)
        makedir(os.path.join(base_dir, body_part, base_new_filename))
        # create each report in the respective folders
        with open(new_filepath, 'w') as w:
            w.write(val)

def remove_start_end(text):
    # remove empty lines, start and end sentences that bear no meaning
    sentences = [s for s in text.splitlines() if s]
    end_word = sentences[-1].split()[0].lower()
    start_word = sentences[2].split()[0].lower()
    if end_word in ['terima', 'atas', 'btk']:
        start_index = 3 if start_word in ['pemeriksaan', 'foto'] else 2
        text = os.linesep.join(sentences[start_index:-1])
    else:
        start_index = 3 if start_word in ['pemeriksaan', 'foto'] else 2
        text = os.linesep.join(sentences[start_index:])
    return text

def remove_punctuations(text):
    punctuations = '''!()[]{};:'"\,<>?@#$%^&*~.''' # / is not removed for fraction, - is not removed for range (ex: l3-l4)
    new_text = ''
    for char in text:
        if char not in punctuations: 
            new_text += char
    return new_text

def remove_unnecessary_symbols(text):

    def to_remove(line):
        line = re.sub(r'^\s*-\s*', '', line)  # Remove dashes at the beginning
        line = re.sub(r'(\s+-\s+|\s+-|-\s+)', ' ', line)  # Remove dashes surrounded by spaces
        line = re.sub(r'\s*-\s*$', '', line)  # Remove dashes at the end
        line = re.sub(r'\s+/\s+', ' ', line) # Remove slashes surrounded by spaces only
        return line.strip()
    
    lines = text.split('\n')

    # Remove strip for every line in the list
    cleaned_lines = [to_remove(line) for line in lines]

    # Join the cleaned lines into a single string
    cleaned_text = '\n'.join(cleaned_lines)

    return cleaned_text

def cleaning(text, base_dir, filename):
    # generate_separate_report(text, base_dir, filename)
    text = text.strip()
    text = text.lower()
    text = remove_punctuations(text)
    text = remove_start_end(text)
    text = remove_unnecessary_symbols(text)
    return text

In [23]:
# generate wordlist file
def generate_wordlist(compiled_text: dict, word_list_dir):
    for key, val in compiled_text.items():
        # remove duplicate words by making the text into set
        val = set(val.split())
        makedir(word_list_dir)
        # write the words into one file -> daatasets_wordlist/bodypart.txt
        with open(os.path.join(word_list_dir, f'{key}.txt'), 'w') as w:
            for word in val:
                w.write(word + '\n')
                    

In [25]:
# directories
base_dir = os.path.join('..','datasets','radiology_clean')
word_list_dir = os.path.join('..', 'datasets_wordlist')
missing_kesan_file = os.path.join('..','report_analysis','all_files_missing_kesan.txt')

with open(missing_kesan_file, 'r', encoding='utf-8') as file:
    missing_kesan_file_list = [line.strip() for line in file]

# body part grouping
body_part_group = [f.lower() for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]
print("body part group:", body_part_group, "\n")

aggregated_report_list = ['BNO-231027-report-1.txt', 'clavicual_kanan-Series_1001_0000-gabung-report-1.txt', 'pelvis-2310022-gabung-report-1.txt', 'pelvis-2310024-report-1.txt', 'pelvis-2310025-report-1.txt', 'pelvis-2310300066-report-1.txt', 'Thorax-231020_Brobkhopneumonia-report-1.txt', 'Thorax-231020_Cardiomegaly-report-1.txt', 'Thorax-231020_Pneumonia-Pneumonia-report-1.txt', 'Thorax-231021_231022-Cardiomegaly-report-1.txt', 'Thorax-231021_231022-diagnosa_gabung-report-3.txt', 'Thorax-231021_231022-Pneumonia-report-2.txt', 'Thorax-231021_231022_Cardiomegaly-Cardiomegaly-report-1.txt', 'Thorax-231021_231022_Pneumonia-Pneumonia-report-1.txt', 'Thorax-231023-Bronkhopneumonia-report-1.txt', 'Thorax-231023-gabung-report-3.txt', 'Thorax-231023-Pneumonia-report-2.txt', 'Thorax-231024-pneumonia-report-2.txt', 'Thorax-231024-report-3.txt', 'Thorax-231025-gabung-report-1.txt', 'Thorax-231025-pneumonia-report-3.txt', 'Thorax-231025-pneumonia_cardiomegaly-report-2.txt', 'Thorax-231026-gabung-report-2.txt', 'Thorax-231026-pneumonia-report-3.txt', 'Thorax-231027-pneumonia-report-3.txt', 'Thorax-231028-pneumonia-report-3.txt', 'Thorax-231028-report-4.txt', 'Thorax-231029-gabung-report-1.txt', 'Thorax-231030-report-1.txt', 'Thorax-231101-report-1.txt', 'Thorax-231103-report-1.txt', 'Thorax-231105-report-1.txt', 'Thorax-231106-report-1.txt', 'Thorax-231107-report-1.txt', 'Thorax-231108-report-1.txt', 'Thorax-231109-report-1.txt', 'Thorax-231110-report-1.txt', 'Thorax-231111-report-1.txt', 'Thorax-231112-report-1.txt', 'Thorax-231113-report-1.txt', 'Thorax-231114-report-1.txt', 'Thorax-231115-report-1.txt', 'Thorax-231116-report-1.txt', 'Thorax-231118-report-1.txt', 'Thorax-231119-report-1.txt', 'Thorax-231120-report-1.txt', 'Thorax-231121-report-1.txt', 'Thorax-231122-report-1.txt', 'Thorax-231123-report-1.txt', 'Thorax-231124-report-1.txt', 'Thorax-231125-report-1.txt', 'Thorax-231126-report-1.txt', 'Thorax-231127-report-1.txt', 'Thorax-231128-report-1.txt', 'Thorax-231129-report-1.txt', 'Thorax-231130-report-1.txt', 'Thorax-231201-report-1.txt', 'Thorax-231202-report-1.txt', 'Thorax-231203-report-1.txt', 'Thorax-231204-report-1.txt', 'Thorax-231205-report-1.txt', 'Thorax-231206-report-1.txt', 'Thorax-231207-report-1.txt', 'Thorax-231208-report-1.txt', 'Thorax-231209-report-1.txt', 'Thorax-231210-report-1.txt', 'Thorax-231211-report-1.txt', 'Thorax-231212-report-1.txt', 'Thorax-231213-report-1.txt', 'Thorax-231214-report-1.txt', 'Thorax-231215-report-1.txt', 'Thorax-231216-report-1.txt', 'Thorax-231217-report-1.txt', 'Thorax-231218-report-1.txt', 'Thorax-231222-report-1.txt', 'Thorax-240101-cardiomegaly-report-2.txt', 'Thorax-240101-report-4.txt', 'Thorax-240102-report-1.txt', 'Thorax-240103-report-1.txt', 'Thorax-240104-report-1.txt', 'Thorax-240105-report-1.txt', 'Thorax-240106-report-1.txt', 'Thorax-240107-report-1.txt', 'Thorax-240108-report-1.txt', 'Thorax-240109-report-1.txt', 'Thorax-240110-report-1.txt', 'Thorax-240111-report-1.txt', 'Thorax-240112-report-1.txt', 'Thorax-240113-report-1.txt', 'Thorax-240114-report-1.txt', 'Thorax-240115-report-1.txt', 'Thorax-240117-report-1.txt', 'Thorax-240119-report-1.txt', 'Thorax-240120-report-1.txt', 'Thorax-240121-report-1.txt', 'Thorax-240122-report-1.txt', 'Thorax-240124-report-1.txt', 'Thorax-240126-report-1.txt', 'Thorax-240127-report-1.txt', 'Thorax-240128-report-1.txt']

# compilation report per body part
# format: {body_part: 'all report text for the body part'}
body_part_text = {}

for root, dirs, files in os.walk(base_dir):
    for file in files:
        # get full filename
        filename = os.path.join(root, file)
        # get body part
        body_part = Path(filename).parent.parent.name
        if body_part in body_part_group:
            # if txt file -> report
            if txt_file(filename):
                if file in missing_kesan_file_list: break
                if file in aggregated_report_list: break
                f = open(filename, "r", encoding="unicode_escape")
                text = f.read()
                text = cleaning(text, base_dir, filename)

                # append text to dict
                if body_part not in body_part_text:
                    body_part_text[body_part] = text + ' '
                else:
                    body_part_text[body_part] += text + ' '
                
generate_wordlist(body_part_text, word_list_dir)

body part group: ['abodemen_3_posisi', 'ankle_bilateral', 'ankle_dextra', 'ankle_joint_bilateral', 'ankle_joint_dextra', 'ankle_sinistra', 'antebrachii', 'appendicogram', 'bahu_dextra', 'bahu_kanan_kiri', 'bahu_sinistra', 'bno', 'calvaria', 'clavicual_kanan', 'cruris', 'cubiti', 'femur_bilateral', 'femur_dextra', 'femur_sinistra', 'genu', 'hip_joint', 'humerus', 'humerus_biateral', 'humerus_dextra', 'humerus_sinistra', 'manus', 'mastoid_bilateral', 'os_sacro_coccygeus', 'pedis_dextra', 'pedis_sinistra', 'pelvis', 'schedel4', 'sinus_paranasal', 'thorax', 'vertebra_cervical', 'vertebra_lumbosakral', 'vertebra_thoracolumbal', 'wrist_joint_bilateral', 'wrist_joint_dextra', 'wrist_joint_sinistra'] 

